In [0]:
%%shell

rm -rf ./torch-scae
git clone https://github.com/bdsaglam/torch-scae
cd ./torch-scae
pip install -q .

In [0]:
%load_ext tensorboard

!mkdir /content/lightning_logs
%tensorboard --logdir '/content/lightning_logs'

In [1]:
import pathlib

import numpy as np
import torch
import torchvision
import torch_scae_experiments
from torch_scae_experiments.mnist.train import train
from pytorch_lightning import seed_everything

seed_everything(42)

42

In [4]:
import hydra
import hydra.experimental
from omegaconf import OmegaConf

In [5]:
import pkg_resources
configs_dir = pkg_resources.resource_filename('torch_scae_experiments', 'configs')

In [6]:
dataset_dir = str(pathlib.Path.home() / 'torch-datasets')

In [19]:
with hydra.experimental.initialize_config_dir(config_dir=str(configs_dir)):
        cfg = hydra.experimental.compose(
            config_name="config",
            # Customize some params
            overrides=[
                f"dataset.directory={dataset_dir}", 
                "trainer.max_epochs=500", # change an existing param
                "+trainer.gpus=1", # add a non-existing param with +
            ]
        )
        OmegaConf.set_struct(cfg, False) # to be able to modify it later
        print(OmegaConf.to_yaml(cfg))
        

seed: 42
data_loader:
  batch_size: 128
  num_workers: 2
dataset:
  directory: /Users/bdsaglam/torch-datasets
meta_optimizer:
  look_ahead: false
  look_ahead_k: 5
  look_ahead_alpha: 0.5
lr_scheduler:
  active: true
  decay_rate: 0.997t
trainer:
  max_epochs: 100
  gradient_clip_val: 0
  gpus: 1
optimizer:
  type: RMSprop
  learning_rate: 3.0e-05
  momentum: 0.9
  weight_decay: 0.0
model:
  image_shape:
  - 1
  - 40
  - 40
  n_classes: 10
  n_obj_caps: 32
  n_part_caps: 40
  ocae_decoder_capsule_params:
    allow_deformations: true
    caps_dropout_rate: 0.0
    dim_caps: 32
    hidden_sizes:
    - 128
    learn_vote_scale: true
    noise_scale: 4.0
    noise_type: uniform
    similarity_transform: false
  ocae_encoder_set_transformer_params:
    dim_hidden: 16
    dim_out: 256
    layer_norm: true
    n_heads: 1
    n_layers: 3
  pcae_cnn_encoder_params:
    activate_final: true
    kernel_sizes:
    - 3
    - 3
    - 3
    - 3
    out_channels:
    - 128
    - 128
    - 128
    - 12

In [12]:
from torch_scae_experiments.mnist.train import train

In [ ]:
torch.cuda.reset_max_memory_cached()
torch.cuda.reset_max_memory_allocated()
torch.cuda.reset_accumulated_memory_stats()

train(cfg)

In [0]:
# resume training
cfg.trainer.update(
    resume_from_checkpoint='/content/lightning_logs/version_0/checkpoints/epoch=99.ckpt',
    max_epochs=700,
)

train(cfg)